<a href="https://colab.research.google.com/github/ashwath-tech/llama-3.2-grumpy-it-finetune/blob/main/Inference/RunFineTunedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. CLEANUP & INSTALL
# We force-reinstall torch and torchvision together to ensure they match
!pip uninstall -y torch torchvision torchaudio transformers peft trl
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers peft datasets bitsandbytes accelerate
print("NOW RESTART THE RUNTIME (Runtime > Restart Session) BEFORE CONTINUING!")

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import userdata
from huggingface_hub import login

try:
    hf_token = userdata.get('HF_TOKEN')
    login(hf_token, add_to_git_credential=True)
except:
    login()

base_model_name = "meta-llama/Llama-3.2-3B-Instruct"
adapter_model_name = "NotSure123/grumpy-llama-3.2-3B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, adapter_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
user_message = input("Ask your question :")
messages = [
    {"role": "system", "content": "You are a highly competent but grumpy and sarcastic IT specialist who provides technically correct answers."},
    {"role": "user", "content": user_message}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=128)
input_length = inputs['input_ids'].shape[1]

generated_tokens = outputs[0][input_length:]

response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response)